In [1]:
!del data.csv
#!rm data.csv

In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
import html



class JobSpider(scrapy.Spider):
    name = "jobs"
    NUM_PAGES = 5

    def start_requests(self):
        urls = [
            'https://www.cwjobs.co.uk/jobs/data-scientist/in-london?radius=10&s=header',
            'https://www.totaljobs.com/jobs/data-scientist/in-london?radius=10'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse1)
            

    def parse1(self, response):
        page_num = response.css(".results-footer-links-container nav span.active::text").get()
        
        next_page = response.url.split("?")[0]+response.css(".results-footer-links-container nav a.btn.btn-default.next::attr(href)").extract()[0]
        
        if int(page_num) <= 5:
            
            
            for job in response.css("div.job-title"):
                yield scrapy.Request(url=str(job.css("a::attr(href)").get()), callback=self.parse2)
                
            yield scrapy.Request(url=next_page, callback=self.parse1)
            
    def parse2(self,response):
        
        company = response.css("section.job-summary li[class='company icon'] div a::text").get()

        description = BeautifulSoup("".join(response.css("div.job-description p").getall()), 'html.parser')
        
        salary_details = response.css("section.job-summary li[class='salary icon'] div::text").get()
        
        job_type = response.css("section.job-summary li[class='job-type icon'] div::text").get()
        
        yield {"URL": response.url, "Company-Recruiter": company, "Description": description.get_text(), "Salary": salary_details, "Job_Type": job_type}

In [3]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'FEED_FORMAT': 'csv',
    'FEED_URI': 'data.csv'
})

process.crawl(JobSpider)
process.start() # the script will block here until the crawling is finished

2020-06-19 08:06:03 [scrapy.utils.log] INFO: Scrapy 2.1.0 started (bot: scrapybot)
2020-06-19 08:06:03 [scrapy.utils.log] INFO: Versions: lxml 4.5.1.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.5 (tags/v3.7.5:5c02a39a0b, Oct 15 2019, 00:11:34) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Windows-10-10.0.18362-SP0
2020-06-19 08:06:03 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-06-19 08:06:03 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2020-06-19 08:06:03 [scrapy.extensions.telnet] INFO: Telnet Password: d497436170eecfb6
2020-06-19 08:06:03 [py.warnings] WARNING: C:\Users\tej78\PycharmProjects\ocean-jobs\venv\lib\site-packages\scrapy\extensions\feedexport.py:210: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FE